# Algoritmo KNN para classificação de Peças de LEGO!

## ⚠️ Atenção
Antes de iniciar a execução, confira o arquivo `src/Settings.py` para garantir que os parâmetros de execução estejam de acordo com o desejado.

**Parâmetros:**
- USE_NUMPY_ARRAY: Se definido como `True`, o programa usará os arrays da biblioteca Numpy para executar os calculos, o que tornará o programa muito mais rápido
- N_ROWS: Número de linhas que será trabalhado nas imagens, tanto para preprocessamento (redução de dimensão) quanto para ler as imagens preprocessadas de volta
- N_COLUMNS: Semelhante ao `N_ROWS`, porém para colunas
- PREPROCESS_IMAGES: Se definido como `True`, o programa irá fazer o preprocessamento das imagens, excluindo qualquer preprocessamento já feito (por ser uma ação destrutiva, o programa irá pedir sua confirmação)
- N_THREADS: Número de threads / processos que será utilizado para fazer processamento paralelo (acelera o tempo de execução)
- N_TESTS_USED: Número de testes para ser usado, caso deseje realizar testes menores (se definido como 0, o programa usará todos os testes)

***Use a célula de imports, variáveis e locks para definir a métrica que será usada.***

## Imports, variáveis e Locks

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time

from threading import Thread
from threading import Lock

import multiprocessing

from sklearn.model_selection import train_test_split 

from src.Images import preprocess_images_from_dir
from src.Images import get_all_images_path
from src.Images import read_processed_image
from src.Images import plot_image
from src.Images import flatten_to_image

from src.Distances import euclidean_distance
from src.Distances import cosine_distance

from src.Knn import Knn

from src.Settings import Settings
from src.Settings import print_all_settings

from src.Parallel import thread_classify_function
from src.Parallel import split_data_into_chunks

METRIC_TO_BE_USED = cosine_distance

file_lock = Lock()
results_lock = Lock()

total_correct = multiprocessing.Value('i', 0)
total_wrong = multiprocessing.Value('i', 0)

print_all_settings()


## Preprocessamento das imagens

In [ ]:
if Settings.PREPROCESS_IMAGES:
    ans = input("Are you sure you want to preprocess all images? [type 'yes']")
    if ans == "yes":
        preprocess_images_from_dir(
            dir_path=Settings.IMAGES_DIR, n_rows=Settings.N_ROWS,
            n_columns=Settings.N_COLUMNS, extension="png"
        )

## Paths para as imagens preprocessadas

In [ ]:
images_path = get_all_images_path(Settings.PROCESSED_IMAGES_PATH)
images_path[:3]

## Organizando os dados

In [ ]:
data = [
    (
        int(path.split(" ")[0].split("/")[-1]),
        read_processed_image(path, Settings.N_ROWS, Settings.N_COLUMNS).flatten()
    ) for path in images_path
] if Settings.USE_NUMPY_ARRAY else [
    (
        int(path.split(" ")[0].split("/")[-1]),
        list(read_processed_image(path, Settings.N_ROWS, Settings.N_COLUMNS).flatten())
    ) for path in images_path
]


y, x = zip(*data)
x = list(x)
y = list(y)

## Divisão em treino e teste

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

## Treinando o modelo (adicionando os elementos conhecidos)

In [ ]:
knn = Knn()

knn.add_points(list(zip(y_train, x_train)))

## Demonstração de imagem preprocessada

In [ ]:
plot_image(flatten_to_image(x_train[0], Settings.N_ROWS, Settings.N_COLUMNS))

## Classificando conjunto de teste

In [ ]:
tests_to_be_used = Settings.N_TESTS_USED if Settings.N_TESTS_USED > 0 else len(x_test)

test_data = list(zip(x_test[:tests_to_be_used], y_test[:tests_to_be_used]))
total_tests = len(test_data)

chunks = split_data_into_chunks(test_data, Settings.N_THREADS)

total_execution_start = time.time()

threads = []

for i, chunk in enumerate(chunks):
    thread = multiprocessing.Process(
        target=thread_classify_function,
        args=(file_lock, results_lock, knn, chunk, total_correct, total_wrong),
        name=f"Thread {i}"
    )
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print(f"Total time for all process: {time.time()-total_execution_start}s")

print(f"{total_correct.value}/{total_tests} correct answers ({total_correct.value/total_tests*100}%)")
print(f"{total_wrong.value}/{total_tests} wrong answers ({total_wrong.value/total_tests*100}%)")